In [1]:
import numpy as np
import time
import timeit

from opt_einsum import contract
import opt_einsum as oe

# 设置BLAS线程数

NUM_THREADS=1

try:
    import mkl
    mkl.set_num_threads(NUM_THREADS)
except ImportError:
    pass
try:
    import blas
    blas.set_num_threads(NUM_THREADS)
except ImportError:
    pass

In [2]:
DIM = 2**10 * 8

np.random.seed(42)
x = np.random.rand(DIM)
y = np.random.rand(DIM)
z = np.random.rand(DIM)
a = 2.5
b = 1.5

In [3]:
# 1. BLAS1 DOT 

def dot_numpy():
    return np.dot(x, y)

def dot_np_einsum():
    return np.einsum('i,i->', x, y)

def dot_oe():
    return oe.contract('i,i->', x, y)

t_np        = timeit.timeit(dot_numpy, number=10000)
t_np_einsum = timeit.timeit(dot_np_einsum, number=10000)
t_oe        = timeit.timeit(dot_oe, number=10000)

print(f"np.dot:         {t_np*1000:.2f} ms (基准)")
print(f"np.einsum:      {t_np_einsum*1000:.2f} ms ({t_np_einsum/t_np:.2f}x)")
print(f"opt_einsum:     {t_oe*1000:.2f} ms ({t_oe/t_np:.2f}x)")

np.dot:         25.99 ms (基准)
np.einsum:      54.22 ms (2.09x)
opt_einsum:     542.59 ms (20.88x)


In [4]:
# 2. BLAS1 NORM 

def norm_numpy():
    return np.linalg.norm(x)

def norm_np_einsum():
    return np.sqrt(np.einsum('i,i->', x, x))

def norm_oe():
    return np.sqrt(oe.contract('i,i->', x, x))

t_np        = timeit.timeit(norm_numpy, number=10000)
t_np_einsum = timeit.timeit(norm_np_einsum, number=10000)
t_oe        = timeit.timeit(norm_oe, number=10000)

print(f"np.linalg.norm: {t_np*1000:.2f} ms (基准)")
print(f"np.einsum:      {t_np_einsum*1000:.2f} ms ({t_np_einsum/t_np:.2f}x)")
print(f"opt_einsum:     {t_oe*1000:.2f} ms ({t_oe/t_np:.2f}x)")

np.linalg.norm: 40.56 ms (基准)
np.einsum:      61.19 ms (1.51x)
opt_einsum:     512.93 ms (12.65x)


In [5]:
# 3. BLAS1 SCAL

def scal_numpy():
    return a * x

def scal_np_einsum():
    return np.einsum(',i->i', a, x)

def scal_oe():
    return oe.contract(',i->i', a, x)

t_np        = timeit.timeit(scal_numpy, number=10000)
t_np_einsum = timeit.timeit(scal_np_einsum, number=10000)
t_oe        = timeit.timeit(scal_oe, number=10000)

print(f"NumPy乘法:      {t_np*1000:.2f} ms (基准)")
print(f"np.einsum:      {t_np_einsum*1000:.2f} ms ({t_np_einsum/t_np:.2f}x)")
print(f"opt_einsum:     {t_oe*1000:.2f} ms ({t_oe/t_np:.2f}x)")

result_np = scal_numpy()
result_np_einsum = scal_np_einsum()
result_oe = scal_oe()
print(f"验证: np.dot vs np.einsum  : {np.allclose(result_np, result_np_einsum)}")
print(f"验证: np.dot vs opt_einsum : {np.allclose(result_np, result_oe)}")

NumPy乘法:      33.49 ms (基准)
np.einsum:      87.38 ms (2.61x)
opt_einsum:     402.48 ms (12.02x)
验证: np.dot vs np.einsum  : True
验证: np.dot vs opt_einsum : True


In [6]:
M, N = 500, 600
A = np.random.rand(M, N)
B = np.random.rand(N, 300)
x = np.random.rand(N)
y = np.random.rand(M)
alpha, beta = 1.5, 2.0

In [7]:
# 5. BLAS2 GEMV

def gemv_numpy():
    return A @ x

def gemv_np_einsum():
    return np.einsum('ij,j->i', A, x)

def gemv_oe():
    return oe.contract('ij,j->i', A, x)

t_np        = timeit.timeit(gemv_numpy, number=2500)
t_np_einsum = timeit.timeit(gemv_np_einsum, number=2500)
t_oe        = timeit.timeit(gemv_oe, number=2500)

print(f"形状: A({M}x{N}), x({N})")
print(f"NumPy @:        {t_np*1000:.2f} ms (基准)")
print(f"np.einsum:      {t_np_einsum*1000:.2f} ms ({t_np_einsum/t_np:.2f}x)")
print(f"opt_einsum:     {t_oe*1000:.2f} ms ({t_oe/t_np:.2f}x)")

形状: A(500x600), x(600)
NumPy @:        110.23 ms (基准)
np.einsum:      204.38 ms (1.85x)
opt_einsum:     252.57 ms (2.29x)


In [8]:
M, N, K = 300, 400, 500
A = np.random.rand(M, N)
B = np.random.rand(N, K)
C = np.random.rand(K, M)
D = np.random.rand(K, 200)
alpha, beta = 1.5, 2.0

In [9]:
# 6. BLAS3 GEMM

def gemm_numpy():
    return A @ B

def gemm_np_einsum():
    return np.einsum('ij,jk->ik', A, B)

def gemm_oe():
    return oe.contract('ij,jk->ik', A, B)

t_np = timeit.timeit(gemm_numpy, number=100)
t_np_einsum = timeit.timeit(gemm_np_einsum, number=100)
t_oe = timeit.timeit(gemm_oe, number=100)

print(f"形状: A({M}x{N}), B({N}x{K}) → ({M}x{K})")
print(f"NumPy @:        {t_np*1000:.2f} ms (基准)")
print(f"np.einsum:      {t_np_einsum*1000:.2f} ms ({t_np_einsum/t_np:.2f}x)")
print(f"opt_einsum:     {t_oe*1000:.2f} ms ({t_oe/t_np:.2f}x)")

形状: A(300x400), B(400x500) → (300x500)
NumPy @:        261.40 ms (基准)
np.einsum:      1252.47 ms (4.79x)
opt_einsum:     276.70 ms (1.06x)


In [10]:
# 7. product of three matrix

def triple_numpy():
    return A @ B @ C

def triple_np_einsum():
    return np.einsum('ij,jk,kl->il', A, B, C)

def triple_np_einsum_opt():
    return np.einsum('ij,jk,kl->il', A, B, C, optimize=True)

def triple_oe_naive():
    # 默认优化
    return oe.contract('ij,jk,kl->il', A, B, C)

expr = oe.contract_expression('ij,jk,kl->il', A.shape, B.shape, C.shape)
def triple_oe_optimized():
    # 预编译表达式
    # expr = oe.contract_expression('ij,jk,kl->il', A.shape, B.shape, C.shape)
    return expr(A, B, C)

t_np           = timeit.timeit(triple_numpy, number=100)
# t_np_einsum    = timeit.timeit(triple_np_einsum, number=5)
t_np_einsum_opt= timeit.timeit(triple_np_einsum_opt, number=100)
t_oe_naive     = timeit.timeit(triple_oe_naive, number=100)
t_oe_optimized = timeit.timeit(triple_oe_optimized, number=100) 

print(f"形状: A({M}x{N}), B({N}x{K}), C({K}x{K}) → ({M}x{K})")
print(f"NumPy @           :        {t_np*1000:.2f} ms (基准)")
# print(f"np.einsum:      {t_np_einsum*1000*20:.2f} ms ({t_np_einsum*20/t_np:.2f}x)")
print(f"np.einsum(opt)    :    {t_np_einsum_opt*1000:.2f} ms ({t_np_einsum_opt/t_np:.2f}x)")
print(f"opt_einsum(naive) : {t_oe_naive*1000:.2f} ms ({t_oe_naive/t_np:.2f}x)")
print(f"opt_einsum(预编译) : {t_oe_optimized*1000:.2f} ms ({t_oe_optimized/t_np:.2f}x)")

形状: A(300x400), B(400x500), C(500x500) → (300x500)
NumPy @           :        454.52 ms (基准)
np.einsum(opt)    :    445.00 ms (0.98x)
opt_einsum(naive) : 480.11 ms (1.06x)
opt_einsum(预编译) : 442.06 ms (0.97x)


In [11]:
# 8. Trace 

N_trace = 5000
A_square = np.random.rand(N_trace, N_trace)

def trace_numpy():
    return np.trace(A_square)

def trace_np_einsum():
    return np.einsum('ii->', A_square)

def trace_oe():
    return oe.contract('ii->', A_square)

t_np        = timeit.timeit(trace_numpy, number=5000)
t_np_einsum = timeit.timeit(trace_np_einsum, number=5000)
t_oe        = timeit.timeit(trace_oe, number=5000)

print(f"形状: A({N_trace}x{N_trace})")
print(f"np.trace:       {t_np*1000:.2f} ms (基准)")
print(f"np.einsum:      {t_np_einsum*1000:.2f} ms ({t_np_einsum/t_np:.2f}x)")
print(f"opt_einsum:     {t_oe*1000:.2f} ms ({t_oe/t_np:.2f}x)")

形状: A(5000x5000)
np.trace:       170.50 ms (基准)
np.einsum:      171.64 ms (1.01x)
opt_einsum:     307.75 ms (1.81x)


In [12]:
# 9. Hadamard product 

M_h, N_h = 300, 400
A_h = np.random.rand(M_h, N_h)
B_h = np.random.rand(M_h, N_h)

def hadamard_numpy():
    return A_h * B_h

def hadamard_np_einsum():
    return np.einsum('ij,ij->ij', A_h, B_h)

def hadamard_oe():
    return oe.contract('ij,ij->ij', A_h, B_h)

t_np        = timeit.timeit(hadamard_numpy, number=500)
t_np_einsum = timeit.timeit(hadamard_np_einsum, number=500)
t_oe        = timeit.timeit(hadamard_oe, number=500)

print(f"形状: A,B({M_h}x{N_h})")
print(f"NumPy *:        {t_np*1000:.2f} ms (基准)")
print(f"np.einsum:      {t_np_einsum*1000:.2f} ms ({t_np_einsum/t_np:.2f}x)")
print(f"opt_einsum:     {t_oe*1000:.2f} ms ({t_oe/t_np:.2f}x)")

形状: A,B(300x400)
NumPy *:        41.01 ms (基准)
np.einsum:      46.01 ms (1.12x)
opt_einsum:     65.84 ms (1.61x)


In [13]:
# 10. outer product 

def outer_numpy():
    return np.outer(x[:100], y[:100])

def outer_np_einsum():
    return np.einsum('i,j->ij', x[:100], y[:100])

def outer_oe():
    return oe.contract('i,j->ij', x[:100], y[:100])

t_np        = timeit.timeit(outer_numpy, number=500)
t_np_einsum = timeit.timeit(outer_np_einsum, number=500)
t_oe        = timeit.timeit(outer_oe, number=500)

print(f"形状: x,y(100) → (100x100)")
print(f"np.outer:       {t_np*1000:.2f} ms (基准)")
print(f"np.einsum:      {t_np_einsum*1000:.2f} ms ({t_np_einsum/t_np:.2f}x)")
print(f"opt_einsum:     {t_oe*1000:.2f} ms ({t_oe/t_np:.2f}x)")

形状: x,y(100) → (100x100)
np.outer:       6.31 ms (基准)
np.einsum:      3.27 ms (0.52x)
opt_einsum:     25.77 ms (4.08x)


In [14]:
# 12. Tensor contraction

I, J, K, L, M = 30, 40, 50, 60, 70
A_3d = np.random.rand(I, J, K)
B_3d = np.random.rand(J, L, M)

def tensor_contract_np_einsum():
    return np.einsum('ijk,jlm->iklm', A_3d, B_3d)

def tensor_contract_np_einsum_opt():
    return np.einsum('ijk,jlm->iklm', A_3d, B_3d, optimize=True)

def tensor_contract_oe_naive():
    return oe.contract('ijk,jlm->iklm', A_3d, B_3d)

def tensor_contract_oe_optimized():
    return oe.contract('ijk,jlm->iklm', A_3d, B_3d, optimize='optimal')

def tensor_contract_oe_greedy():
    return oe.contract('ijk,jlm->iklm', A_3d, B_3d, optimize='greedy')

# 预编译
expr_tensor = oe.contract_expression('ijk,jlm->iklm', A_3d.shape, B_3d.shape)
def tensor_contract_oe_precomp():
    return expr_tensor(A_3d, B_3d)

t_np_einsum = timeit.timeit(tensor_contract_np_einsum, number=20)
t_np_einsum_opt = timeit.timeit(tensor_contract_np_einsum_opt, number=20)
t_oe_naive = timeit.timeit(tensor_contract_oe_naive, number=20)
t_oe_optimal = timeit.timeit(tensor_contract_oe_optimized, number=20)
t_oe_greedy = timeit.timeit(tensor_contract_oe_greedy, number=20)
t_oe_precomp = timeit.timeit(tensor_contract_oe_precomp, number=20)

print(f"形状: A({I}x{J}x{K}), B({J}x{L}x{M}) → ({I}x{K}x{L}x{M})")
print(f"np.einsum:      {t_np_einsum*1000:.2f} ms (基准)")
print(f"np.einsum(opt): {t_np_einsum_opt*1000:.2f} ms ({t_np_einsum_opt/t_np_einsum:.2f}x)")
print(f"opt_einsum(无优化): {t_oe_naive*1000:.2f} ms ({t_oe_naive/t_np_einsum:.2f}x)")
print(f"opt_einsum(最优): {t_oe_optimal*1000:.2f} ms ({t_oe_optimal/t_np_einsum:.2f}x)")
print(f"opt_einsum(贪婪): {t_oe_greedy*1000:.2f} ms ({t_oe_greedy/t_np_einsum:.2f}x)")
print(f"opt_einsum(预编译): {t_oe_precomp*1000:.2f} ms ({t_oe_precomp/t_np_einsum:.2f}x)")

形状: A(30x40x50), B(40x60x70) → (30x50x60x70)
np.einsum:      8786.58 ms (基准)
np.einsum(opt): 438.80 ms (0.05x)
opt_einsum(无优化): 391.82 ms (0.04x)
opt_einsum(最优): 354.96 ms (0.04x)
opt_einsum(贪婪): 343.12 ms (0.04x)
opt_einsum(预编译): 361.94 ms (0.04x)


In [15]:
# 13. batch GEMM

batch_size = 100
B1, B2, B3 = 10, 20, 30
A_batch = np.random.rand(batch_size, B1, B2)
B_batch = np.random.rand(batch_size, B2, B3)

def batch_matmul_numpy():
    return np.stack([A_batch[i] @ B_batch[i] for i in range(batch_size)])

def batch_matmul_np_einsum():
    return np.einsum('bij,bjk->bik', A_batch, B_batch)

def batch_matmul_oe():
    return oe.contract('bij,bjk->bik', A_batch, B_batch)

t_np = timeit.timeit(batch_matmul_numpy, number=20)
t_np_einsum = timeit.timeit(batch_matmul_np_einsum, number=20)
t_oe = timeit.timeit(batch_matmul_oe, number=20)

print(f"形状: A({batch_size}x{B1}x{B2}), B({batch_size}x{B2}x{B3})")
print(f"NumPy循环:      {t_np*1000:.2f} ms (基准)")
print(f"np.einsum:      {t_np_einsum*1000:.2f} ms ({t_np_einsum/t_np:.2f}x)")
print(f"opt_einsum:     {t_oe*1000:.2f} ms ({t_oe/t_np:.2f}x)")

形状: A(100x10x20), B(100x20x30)
NumPy循环:      6.54 ms (基准)
np.einsum:      11.71 ms (1.79x)
opt_einsum:     12.73 ms (1.95x)


总结: 
1. 对于简单操作(向量点积、矩阵乘法等)，原生NumPy通常最快
2. numpy.einsum 在简单表达式中通常比opt_einsum快
3. numpy.einsum 必须开 Optimize=True 选项
4. opt_einsum 在复杂表达式(三重以上乘法)和优化后有明显优势
5. 预编译表达式可以提高opt_einsum性能
6. 对于批量操作和张量缩并，einsum表示法更简洁高效